In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_data_dir = '/kaggle/input/partial-reid/Partial_REID/train'  
test_data_dir = '/kaggle/input/partial-reid/Partial_REID/test'    
input_shape = (150, 150, 3)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical' 
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=False 
)
num_classes = train_generator.num_classes


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) 


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, steps_per_epoch=train_generator.n // batch_size, epochs=200)

In [ ]:
evaluation = model.evaluate(train_generator, steps=len(train_generator))
train_loss, train_accuracy = evaluation[0], evaluation[1]

print(f'Train Accuracy: {train_accuracy * 100:.2f}%')

In [ ]:
evaluation = model.evaluate(test_generator, steps=len(test_generator))
test_loss, test_accuracy = evaluation[0], evaluation[1]

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
import math

def plot_person_images(person_id):
    occluded_images = []
    non_occluded_images = []
    predictions = []
    
    for i in range(len(test_generator)):
        batch = test_generator[i]
        images, labels = batch
        predicted_classes = model.predict(images)
        
        occluded_indices = np.where(labels[:, person_id] == 1)[0]
        occluded_images.extend(images[occluded_indices])
        non_occluded_indices = np.where(labels[:, person_id] == 0)[0]
        non_occluded_images.extend(images[non_occluded_indices])
        predictions.extend(np.argmax(predicted_classes, axis=1))
        
    total_images = len(occluded_images) + len(non_occluded_images)
    cols = 10 
    rows = math.ceil(total_images / cols)
    
    plt.figure(figsize=(20, 20))  
    for i, img in enumerate(occluded_images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img, interpolation='nearest', aspect='auto')  
        plt.title('Occluded', fontsize=12, y=1.05) 
        plt.axis('off')
    
    for i, img in enumerate(non_occluded_images):
        plt.subplot(rows, cols, len(occluded_images) + i + 1)
        plt.imshow(img, interpolation='nearest', aspect='auto') 
        plt.title('Non-Occluded', fontsize=12, y=1.05) 
        plt.axis('off')

    plt.tight_layout()
    plt.show()

random_person_id = np.random.randint(0, num_classes) 
plot_person_images(random_person_id)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

def classify_and_display_image(image_filename, model):
    img = image.load_img(image_filename, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    
    if predicted_class == 1:
        title = 'OCCLUDED'
    else:
        title = 'NONOCCLUDED'
    
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()

image_filename = '/kaggle/input/input-image/002_03.jpg' 
classify_and_display_image(image_filename, model)


In [ ]:
training_accuracy = []
epochs = 100  

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
 
    history = model.fit(train_generator, steps_per_epoch=train_generator.n // batch_size, epochs=1, verbose=1)
    evaluation = model.evaluate(train_generator, steps=len(train_generator), verbose=0)
    train_accuracy = evaluation[1]
    training_accuracy.append(train_accuracy)
    print(f'Training Accuracy for Epoch {epoch + 1}: {train_accuracy * 100:.2f}%')

plt.figure(figsize=(8, 6))
plt.plot(range(1, epochs + 1), training_accuracy, label='Training Accuracy', marker='o')
plt.title('Training Accuracy vs Number of Epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.save('Occlusion_Classification_Dataset-1.h5')